In [35]:
import numpy as np
import pandas as pd
from pandas.api.types import is_string_dtype, is_object_dtype
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from cvxopt import matrix, solvers
from sklearn.neighbors import KDTree
from typing import List, Tuple, Optional, Union

import sys
import os
import platform

if platform.system() == 'Windows':
    build_path = os.path.join(os.getcwd(), "build", "Debug")
elif platform.system() == 'Darwin':  # macOS
    build_path = os.path.join(os.getcwd(), "build")
else:
    raise EnvironmentError("Unsupported operating system")

sys.path.append(build_path)
 
import ocsvm

In [36]:
# Function to identify unique ID column
def identify_unique_id(dataframe: pd.DataFrame, filename: str) -> pd.DataFrame:
    continuous_ratio_threshold = 2
    unique_id_column = []
    for col in dataframe.columns:
        if (
            dataframe[col].nunique() == len(dataframe)  # Unique values equal to the number of rows
            and not pd.api.types.is_float_dtype(dataframe[col])  # Exclude float columns
        ):
            # Calculate range-to-unique ratio for integer columns
            if pd.api.types.is_integer_dtype(dataframe[col]):
                col_range = dataframe[col].max() - dataframe[col].min() + 1
                if col_range / dataframe[col].nunique() > continuous_ratio_threshold:
                    continue  # Likely a continuous integer column, not an ID
            unique_id_column = col
    if unique_id_column:
        dataframe = dataframe.drop(columns=unique_id_column)
    elif filename == 'Datasets/bank-customer-churn-prediction.csv':
        dataframe = dataframe.drop(columns="customer_id")
    return dataframe

In [37]:
def convert_dtype(dataframe: pd.DataFrame) -> pd.DataFrame:
    for column in dataframe.columns:
        if is_object_dtype(dataframe[column]):
            unique_values = dataframe[column].unique()
            value_to_float = {value: float(index) for index, value in enumerate(unique_values)}
            dataframe[column] = dataframe[column].map(value_to_float)
        if is_string_dtype(dataframe[column]):
            unique_values = dataframe[column].unique()
            value_to_float = {value: float(index) for index, value in enumerate(unique_values)}
            dataframe[column] = dataframe[column].map(value_to_float)
    return dataframe

In [38]:
def preprocessing(dataframe: pd.DataFrame, filename: str) -> pd.DataFrame:
    dataframe = identify_unique_id(dataframe, filename)
    dataframe = convert_dtype(dataframe)
    return dataframe

In [39]:
def corr_matrix(dataframe: pd.DataFrame) -> None:
    preprocessing(dataframe)
    correlation_matrix = dataframe.corr()
    plt.figure(figsize=(10, 5))
    sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
    plt.show()

In [40]:
class DBSCAN:
    def __init__(self, eps=1.0, minPts=5) -> None:
        self.eps = eps  # Radius for neighbourhood search
        self.minPts = minPts  # Minimum number of points to form a cluster

    def fit(self, dataframe: np.ndarray) -> List[List[int]]:
        # Initialise labels array with -1 (no cluster assigned)
        self.labels = np.zeros(len(dataframe), dtype=int) - 1
        self.clusters = []  # List to hold all clusters
        cidx = 0  # Cluster ID counter

        # Build a KDTree for fast nearest-neighbour search
        tree = KDTree(dataframe)

        for x in range(len(dataframe)):
            if self.labels[x] == -1:
                neighbours = self.get_neighbours(tree, dataframe, x)
                if len(neighbours) < self.minPts:
                    self.labels[x] = 0  # Mark point as noise (anomaly)
                else:
                    cidx += 1
                    self.clusters.append([x])
                    self.labels[x] = cidx  # Assign cluster ID to the point

                    # Expand the cluster with points
                    to_visit = neighbours.copy()
                    while to_visit:
                        y = to_visit.pop()
                        if self.labels[y] == -1:  # If point is unvisited
                            self.labels[y] = cidx
                            self.clusters[cidx - 1].append(y)
                            neighbours2 = self.get_neighbours(tree, dataframe, y)
                            if len(neighbours2) >= self.minPts:
                                to_visit.extend(neighbours2)
                        elif self.labels[y] == 0:  # If point is marked as noise
                            self.labels[y] = cidx
                            self.clusters[cidx - 1].append(y)

        return self.clusters

    def get_neighbours(self, tree: KDTree, dataframe: np.ndarray, x: int) -> List[int]:
        # Use KDTree to find neighbours within eps distance
        indices = tree.query_radius([dataframe[x]], r=self.eps)[0]
        return indices.tolist()

    def plot(self, data: np.ndarray) -> None:
        # Create a 2D plot (instead of 3D)
        plt.figure(figsize=(10, 8))

        # Find noise points and clusters
        noise_idx = np.where(self.labels == 0)[0]
        cluster_idxs = [np.array(c) for c in self.clusters]

        # Assign colours to clusters
        colours = cm.rainbow(np.linspace(0, 1, len(self.clusters) + 1))

        # Plot each cluster
        for i, cluster_idx in enumerate(cluster_idxs):
            plt.scatter(data[cluster_idx, 0], data[cluster_idx, 1], 
                        color=colours[i], s=10, label=f'Cluster {i + 1}')

        # Plot anomalies
        if len(noise_idx) > 0:
            plt.scatter(data[noise_idx, 0], data[noise_idx, 1], 
                        color='black', s=10, label='Anomalies')

        # Add labels and legend
        plt.title("DBSCAN Clustering")
        plt.xlabel("Feature 1")
        plt.ylabel("Feature 2")
        plt.legend()
        plt.show()

In [41]:
def ocsvm_index_list(model, data: List[Tuple[int, float, float]]) -> List[int]:
    inliers_data = []
    outliers_data = []
    
    for point, label in zip(data, [model.predict(point[1:]) for point in data]):
        if label == 1:  # Inlier
            inliers_data.append(point)
        else:  # Outlier
            outliers_data.append(point)

    index_list = [index[0] for index in outliers_data]

    return index_list

In [42]:
def ocsvm_plot(model, data: List[Tuple[int, float, float]]) -> None:
    inliers_data = []
    outliers_data = []
    
    for point, label in zip(data, [model.predict(point[1:]) for point in data]):
        if label == 1:  # Inlier
            inliers_data.append(point)
        else:  # Outlier
            outliers_data.append(point)
            
    # Plot inliers and outliers
    inliers_x = [point[1] for point in inliers_data]
    inliers_y = [point[2] for point in inliers_data]
    outliers_x = [point[1] for point in outliers_data]
    outliers_y = [point[2] for point in outliers_data]
    
    plt.figure(figsize=(10, 6))
    plt.scatter(inliers_x, inliers_y, c='blue', label='Inliers', marker='o')
    plt.scatter(outliers_x, outliers_y, c='red', label='Outliers', marker='x')
    plt.title('Inliers and Outliers')
    plt.xlabel('Previous qualification (grade)')
    plt.ylabel('Admission grade')
    plt.legend()
    plt.grid(True)
    plt.show()

In [43]:
class IsolationTree:
    def __init__(self, max_depth: int) -> None:
        self.max_depth = max_depth
        self.split_feature = None
        self.split_value = None
        self.left = None
        self.right = None
        self.is_terminal = False
        self.size = 0

    def fit(self, X: np.ndarray, depth: int = 0) -> None:
        # Check the termination condition
        if depth >= self.max_depth or len(X) <= 1:
            self.is_terminal = True
            self.size = len(X)
            return

        # Randomly choose a feature and split value
        num_features = X.shape[1]
        self.split_feature = np.random.randint(0, num_features)
        min_val, max_val = np.min(X[:, self.split_feature]), np.max(X[:, self.split_feature])

        if min_val == max_val:  # No meaningful split
            self.is_terminal = True
            self.size = len(X)
            return

        self.split_value = np.random.uniform(min_val, max_val)

        # Partition the data
        left_mask = X[:, self.split_feature] < self.split_value # Mask for left subset
        right_mask = ~left_mask # Mask for right subset

        self.left = IsolationTree(self.max_depth)
        self.right = IsolationTree(self.max_depth)

        self.left.fit(X[left_mask], depth + 1)
        self.right.fit(X[right_mask], depth + 1)

    def path_length(self, X: np.ndarray) -> np.ndarray:
        path_lengths = np.zeros(X.shape[0])

        for i, x in enumerate(X):
            node = self
            depth = 0
            while not node.is_terminal: # Traverse tree until termianl node reached
                if x[node.split_feature] < node.split_value:
                    node = node.left
                else:
                    node = node.right
                depth += 1
            path_lengths[i] = depth + self._c(node.size)
        return path_lengths

    @staticmethod
    def _c(size: int) -> float:
        # Calculate average path length of a binary tree for given size
        if size <= 1:
            return 0
        return 2 * (np.log(size - 1) + 0.5772156649) - (2 * (size - 1) / size)


class IsolationForest:
    def __init__(self, n_estimators: int = 100, max_samples: int = 256, max_depth: Optional[int] = None) -> None:
        self.n_estimators = n_estimators
        self.max_samples = max_samples
        self.max_depth = max_depth
        self.forest = []

    def fit(self, X: Union[np.ndarray, pd.DataFrame]) -> None:
        # Convert DataFrame to NumPy array if needed
        if isinstance(X, pd.DataFrame):
            X = X.values

        num_samples, num_features = X.shape
        self.max_samples = min(self.max_samples, num_samples)
        if self.max_depth is None:
            self.max_depth = int(np.ceil(np.log2(self.max_samples)))

        self.forest = []
        for _ in range(self.n_estimators):
            sample_indices = np.random.choice(num_samples, self.max_samples, replace=False)
            X_sample = X[sample_indices]
            tree = IsolationTree(self.max_depth)
            tree.fit(X_sample)
            self.forest.append(tree)

    def anomaly_score(self, X: Union[np.ndarray, pd.DataFrame]) -> np.ndarray:
        # Convert DataFrame to NumPy array if needed
        if isinstance(X, pd.DataFrame):
            X = X.values

        path_lengths = np.zeros(X.shape[0])
        for tree in self.forest:
            path_lengths += tree.path_length(X)
        path_lengths /= len(self.forest)

        # Compute anomaly score based on path lengths
        c_n = 2 * (np.log(self.max_samples - 1) + 0.5772156649) - (2 * (self.max_samples - 1) / self.max_samples)
        scores = 2 ** (-path_lengths / c_n)
        return scores

    def predict(self, X: Union[np.ndarray, pd.DataFrame], threshold: float = 0.5) -> np.ndarray:
        scores = self.anomaly_score(X)
        return np.where(scores > threshold, -1, 1)  # -1 for anomalies, 1 for normal

    def plot(self, X: Union[np.ndarray, pd.DataFrame], predictions: np.ndarray, feature_names: Optional[List[str]] = None) -> None:
        if isinstance(X, pd.DataFrame):
            X = X.values

        # Assign colors for anomalies and inliers
        colors = np.array(['blue' if p == 1 else 'red' for p in predictions])
        plt.figure(figsize=(10, 6))

        if X.shape[1] == 2:  # If data has 2 features
            plt.scatter(X[:, 0], X[:, 1], c=colors, alpha=0.8)
            plt.xlabel(feature_names[0])
            plt.ylabel(feature_names[1])
        else:  # For more than 2 features, plot only the first two
            plt.scatter(X[:, 0], X[:, 1], c=colors, alpha=0.8)
            plt.xlabel('Feature 1')
            plt.ylabel('Feature 2')
        plt.title('Isolation Forest - Anomaly Detection')
        plt.legend(handles=[plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='Inliers'),
                            plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='Anomalies')],
                   loc='upper right')
        plt.grid()
        plt.show()

In [44]:
def drop_OCSVM(dataframe: pd.DataFrame, filename: str, nu: float, gamma: float) -> pd.DataFrame:
    used_cols = [col for col in dataframe.columns if dataframe[col].nunique() > 50]
    print(used_cols)
    for i in range(len(used_cols)):
        for j in range(i+1, len(used_cols)):
            data = [(index, row[used_cols[i]], row[used_cols[j]]) 
                    for index, row in dataframe.iterrows() 
                    if not pd.isna(row[used_cols[i]]) and not pd.isna(row[used_cols[j]])]
            # Train custom One-Class SVM
            oc_svm = ocsvm.OCSVM(nu=nu, kernel="rbf", gamma=0.9)
            oc_svm.fit([point[1:] for point in data])
            index_list = ocsvm_index_list(oc_svm, data)
            dataframe.loc[dataframe.index[index_list], used_cols[i]] = np.nan
            dataframe.loc[dataframe.index[index_list], used_cols[j]] = np.nan
    dataframe.to_csv(filename, index=False)
    return dataframe

In [45]:
def drop_DBSCAN(dataframe: pd.DataFrame, filename: str) -> pd.DataFrame:
    used_cols = [col for col in dataframe.columns if dataframe[col].nunique() > 50]
    print(used_cols)
    for i in range(len(used_cols)):
        for j in range(i+1, len(used_cols)):
            data = dataframe[[used_cols[i], used_cols[j]]].dropna()
            data_norm = StandardScaler().fit_transform(data)
    
            # Train custom DBSCAN
            dbscan = DBSCAN(eps=4, minPts=5)
            dbscan.fit(data_norm)
            dbscan.plot(data_norm)
            outlier_ids = np.where(dbscan.labels == 0)[0]
            dataframe.loc[dataframe.index[outlier_ids], used_cols[i]] = np.nan
            dataframe.loc[dataframe.index[outlier_ids], used_cols[j]] = np.nan
    dataframe.to_csv(filename, index=False)
    return dataframe

In [46]:
def drop_IF(dataframe: pd.DataFrame, filename: str, threshold: float) -> pd.DataFrame:
    used_cols = [col for col in dataframe.columns if dataframe[col].nunique() > 50]
    print(used_cols)
    for i in range(len(used_cols)):
        for j in range(i+1, len(used_cols)):
            data = dataframe[[used_cols[i], used_cols[j]]].dropna()

            # Train Isolation Forest algorithm
            iforest = IsolationForest(n_estimators=100, max_samples=len(data), max_depth=100)
            iforest.fit(data)
            predictions = iforest.predict(data, threshold=threshold)
            iforest.plot(data, predictions, feature_names=['Absences', 'GPA'])
            
            outlier_ids = np.where(predictions == -1)[0]
            print(outlier_ids)
            dataframe.loc[dataframe.index[outlier_ids], used_cols[i]] = np.nan
            dataframe.loc[dataframe.index[outlier_ids], used_cols[j]] = np.nan
    dataframe.to_csv(filename, index=False)
    return dataframe